In [1]:
# 基本配置
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
from langchain_community.embeddings.cloudflare_workersai import CloudflareWorkersAIEmbeddings
from supabase.client import Client, create_client

load_dotenv(override=True)

qw_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('DASHSCOPE_API_BASE'),
    openai_api_key=os.getenv('DASHSCOPE_API_KEY'),
    model_name="qwen2-1.5b-instruct",
    temperature=0,
    streaming=True,
)
embedding = CloudflareWorkersAIEmbeddings(
    account_id=os.getenv('CF_ACCOUNT_ID'),
    api_token=os.getenv('CF_API_TOKEN'),
    model_name="@cf/baai/bge-small-en-v1.5",
)

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 让我们看一些例子来看看它是如何工作的。首先，我们构造一个runnable（它接受一个dict作为输入，并返回一个消息作为输出）：
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who's good at {ability}. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
runnable = prompt | qw_llm_openai

In [4]:
# runnable.invoke("ability", "chatting")

TypeError: Expected mapping type as input to ChatPromptTemplate. Received <class 'str'>.

In [ ]:
# 查看内存集成页面，了解使用Redis和其他提供商实现的聊天消息历史记录。
# 在这里，我们演示了如何使用内存中的 ChatMessageHistory 以及如何使用更持久的存储 RedisChatMessageHistory 。

In-memory 内存中

我们构造了一个可调用的 get_session_history ，它引用这个dict来返回 ChatMessageHistory 的一个实例。可以通过在运行时将配置传递给 RunnableWithMessageHistory 来指定可调用对象的参数。默认情况下，配置参数应为单个字符串 session_id 。这可以通过 history_factory_config kwarg进行调整。

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

请注意，我们已经指定了 input_messages_key （作为最新输入消息的键）和 history_messages_key （添加历史消息的键）。

当调用这个新的runnable时，我们通过配置参数指定相应的聊天记录：

In [6]:
with_message_history.invoke(
    {"ability": "math", "input": "What does cosine mean?"},
    config={"configurable": {"session_id": "abc123"}},
)

2024-07-05 11:20:50,473:INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='Cosine is a trigonometric function of an angle that measures the ratio of the adjacent side to the hypotenuse in a right triangle.', response_metadata={'finish_reason': 'stop', 'model_name': 'qwen2-1.5b-instruct'}, id='run-a8c15ee5-4150-498b-897d-f5008d308939-0')

In [7]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What does cosine mean?'), AIMessage(content='Cosine is a trigonometric function of an angle that measures the ratio of the adjacent side to the hypotenuse in a right triangle.', response_metadata={'finish_reason': 'stop', 'model_name': 'qwen2-1.5b-instruct'}, id='run-a8c15ee5-4150-498b-897d-f5008d308939-0')])}

In [8]:
# Remembers
with_message_history.invoke(
    {"ability": "math", "input": "What?"},
    config={"configurable": {"session_id": "abc123"}},
)

2024-07-05 11:22:32,506:INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content="It seems like you might have misunderstood my response. I explained what cosine means: it's a trigonometric function used to find the ratio of the adjacent side to the hypotenuse in a right-angled triangle, which helps solve for angles and distances in geometry.", response_metadata={'finish_reason': 'stop', 'model_name': 'qwen2-1.5b-instruct'}, id='run-38d07bec-4190-4a4f-aa1c-89392ad2fbd8-0')

In [9]:
# New session_id --> does not remember.
with_message_history.invoke(
    {"ability": "math", "input": "What?"},
    config={"configurable": {"session_id": "def234"}},
)

2024-07-05 11:22:48,182:INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content="I'm a language model, I can't answer questions that require complex calculations.", response_metadata={'finish_reason': 'stop', 'model_name': 'qwen2-1.5b-instruct'}, id='run-46754ebe-0a18-4e11-8172-37122e814a6a-0')

我们用来跟踪消息历史的配置参数可以通过向 history_factory_config 参数传递一个 ConfigurableFieldSpec 对象列表来定制。
下面，我们使用两个参数： user_id 和 conversation_id 。

In [10]:
from langchain_core.runnables import ConfigurableFieldSpec

store = {}


def get_session_history(user_id: str, conversation_id: str) -> BaseChatMessageHistory:
    if (user_id, conversation_id) not in store:
        store[(user_id, conversation_id)] = ChatMessageHistory()
    return store[(user_id, conversation_id)]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",
            annotation=str,
            name="Conversation ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
)

In [14]:
with_message_history.invoke(
    {"ability": "math", "input": "who are you"},
    config={"configurable": {"user_id": "123", "conversation_id": "1"}},
)

2024-07-05 11:25:13,121:INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='I am a language model designed to answer questions and provide information on various topics.', response_metadata={'finish_reason': 'stop', 'model_name': 'qwen2-1.5b-instruct'}, id='run-c42dff8f-9402-4963-8bc0-5606a0dfde8a-0')

In [15]:
with_message_history.invoke(
    {"ability": "math", "input": "what？"},
    config={"configurable": {"user_id": "123", "conversation_id": "1"}},
)

2024-07-05 11:25:17,104:INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='I am a language model that can answer questions and provide information on various topics.', response_metadata={'finish_reason': 'stop', 'model_name': 'qwen2-1.5b-instruct'}, id='run-a2efd0d2-a5fc-4511-a391-845cb128893e-0')

In [16]:
with_message_history.history_messages_key

'history'

In [17]:
store

{('123',
  '1'): InMemoryChatMessageHistory(messages=[HumanMessage(content='Hello'), AIMessage(content='Hello! How can I assist you today?', response_metadata={'finish_reason': 'stop', 'model_name': 'qwen2-1.5b-instruct'}, id='run-023d8283-edba-4558-af19-db5f66654301-0'), HumanMessage(content='what？'), AIMessage(content='What do you need help with?', response_metadata={'finish_reason': 'stop', 'model_name': 'qwen2-1.5b-instruct'}, id='run-5f34a0c0-4de9-464b-92d7-d01d1bc05b3f-0'), HumanMessage(content='who are you'), AIMessage(content='I am a language model designed to answer questions and provide information on various topics.', response_metadata={'finish_reason': 'stop', 'model_name': 'qwen2-1.5b-instruct'}, id='run-c42dff8f-9402-4963-8bc0-5606a0dfde8a-0'), HumanMessage(content='what？'), AIMessage(content='I am a language model that can answer questions and provide information on various topics.', response_metadata={'finish_reason': 'stop', 'model_name': 'qwen2-1.5b-instruct'}, id='ru

上面的runnable接受一个dict作为输入，并返回一个BaseMessage。下面我们展示一些替代品。

In [20]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableParallel
from operator import itemgetter


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


# with_message_history = RunnableWithMessageHistory(
#     itemgetter("input_messages") | qw_llm_openai,
#     get_session_history,
#     input_messages_key="input_messages",
# )

chain = RunnableParallel({"output_message": qw_llm_openai})
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    output_messages_key="output_message",
)

with_message_history.invoke(
    [HumanMessage(content="我今天有点精疲力尽，怎办？")],
    config={"configurable": {"session_id": "baz"}},
)

AssertionError: The input to RunnablePassthrough.assign() must be a dict.

In [19]:
with_message_history.invoke(
    [HumanMessage(content="还有呢")],
    config={"configurable": {"session_id": "baz"}},
)

2024-07-05 11:29:00,458:INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


{'output_message': AIMessage(content='除了上述建议之外，还可以考虑：\n\n1. 调整作息时间：尽量让自己早睡早起，保持规律的生活习惯，让自己的生物钟得到调整。\n\n2. 做一些自己喜欢的事情：做一些自己感兴趣的事情，比如听音乐、看电影、阅读书籍或者画画等，都可以帮助你暂时忘记疲劳的感觉。\n\n3. 打游戏：玩一些轻松的游戏也可以帮助你放松心情，同时也能锻炼大脑，提高思维能力。\n\n4. 学习新的技能：学习一项新技能，如编程、烹饪、绘画等等，不仅可以提高自我价值感，还能帮助你摆脱无聊和乏味的情绪。\n\n5. 多与朋友交流：与家人、朋友多沟通，分享彼此的经历和感受，可以让你感受到更多的温暖和支持，从而缓解疲劳感。\n\n6. 减少电子设备使用：过度依赖电子设备可能会导致焦虑和抑郁，所以尽量减少电子设备的使用时间，给自己留出更多的时间去思考和探索。\n\n7. 培养兴趣爱好：培养一些个人的兴趣爱好，比如写作、摄影、园艺等等，不仅可以丰富生活，还能让你有更多的乐趣和满足感。\n\n总之，疲劳感是一种常见的情绪反应，可以通过多种方式来缓解。希望以上建议对你有所帮助！', response_metadata={'finish_reason': 'stop', 'model_name': 'qwen2-1.5b-instruct'}, id='run-c3857043-049f-4bc0-8c56-bee82c7ab516-0')}

In [ ]:
RunnableWithMessageHistory(
    qw_llm_openai,
    get_session_history,
)

In [ ]:
from operator import itemgetter

RunnableWithMessageHistory(
    itemgetter("input_messages") | qw_llm_openai,
    get_session_history,
    input_messages_key="input_messages",
)

Persistent storage 持久存储

在许多情况下，最好保存会话历史。 RunnableWithMessageHistory 不知道 get_session_history 可调用对象如何检索其聊天消息历史。
这里有一个使用本地文件系统的例子。下面我们将演示如何使用Redis。查看内存集成页面，了解使用其他提供商实现的聊天消息历史记录。

In [21]:
# %pip install --upgrade --quiet redis


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
REDIS_URL = "redis://192.168.22.238:6379/0"

更新消息历史实现只需要我们定义一个新的可调用对象，这次返回一个 RedisChatMessageHistory 的实例：

In [23]:
from langchain_community.chat_message_histories import RedisChatMessageHistory


def get_message_history(session_id: str) -> RedisChatMessageHistory:
    return RedisChatMessageHistory(session_id, url=REDIS_URL)


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_message_history,
    input_messages_key="input",
    history_messages_key="history",
)

我们可以像以前一样调用：

In [24]:
with_message_history.invoke(
    {"ability": "math", "input": "What does cosine mean?"},
    config={"configurable": {"session_id": "foobar"}},
)

2024-07-05 11:37:04,240:INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='Cosine is a trigonometric function of an angle that measures the ratio of the adjacent side to the hypotenuse in a right triangle.', response_metadata={'finish_reason': 'stop', 'model_name': 'qwen2-1.5b-instruct'}, id='run-8c86640d-336b-4f80-ab59-34a754984cb4-0')

In [25]:
with_message_history.invoke(
    {"ability": "math", "input": "What's its inverse"},
    config={"configurable": {"session_id": "foobar"}},
)

2024-07-05 11:38:06,144:INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='The inverse of cosine, denoted as arccos, returns the angle whose cosine equals a given value.', response_metadata={'finish_reason': 'stop', 'model_name': 'qwen2-1.5b-instruct'}, id='run-cfef9e38-a924-4ed2-8cca-e8886c4b815d-0')

In [26]:
store

{('123',
  '1'): InMemoryChatMessageHistory(messages=[HumanMessage(content='Hello'), AIMessage(content='Hello! How can I assist you today?', response_metadata={'finish_reason': 'stop', 'model_name': 'qwen2-1.5b-instruct'}, id='run-023d8283-edba-4558-af19-db5f66654301-0'), HumanMessage(content='what？'), AIMessage(content='What do you need help with?', response_metadata={'finish_reason': 'stop', 'model_name': 'qwen2-1.5b-instruct'}, id='run-5f34a0c0-4de9-464b-92d7-d01d1bc05b3f-0'), HumanMessage(content='who are you'), AIMessage(content='I am a language model designed to answer questions and provide information on various topics.', response_metadata={'finish_reason': 'stop', 'model_name': 'qwen2-1.5b-instruct'}, id='run-c42dff8f-9402-4963-8bc0-5606a0dfde8a-0'), HumanMessage(content='what？'), AIMessage(content='I am a language model that can answer questions and provide information on various topics.', response_metadata={'finish_reason': 'stop', 'model_name': 'qwen2-1.5b-instruct'}, id='ru